# CSV to SQL

In [284]:
import pandas as pd
import numpy as np
import sys

In [285]:
def DataframeToCsv(dataframe, fileName, tableName, columns):
    numRow = len(dataframe.index)
    numCol = len(dataframe.columns)
    with open(fileName, 'w') as f:
        for row in range(0, numRow):
            print("INSERT INTO " + tableName + " " + columns + " VALUES (", file=f, end="")
            for column in range(0, numCol):
                print("NULL" if (pais.iat[row, column] == "'nan'") else pais.iat[row, column], file=f, end="")
                if (column != numCol-1):
                    print(", ", file=f, end="")
                else:
                    print(");", file=f)

In [286]:
# Reads CSV
pais = pd.read_csv("raw-data/pais.csv")
capital = pd.read_csv("raw-data/List_of_national_capitals_1.csv")
# Treats CSV
pais.drop("Source and date", axis=1, inplace=True)
pais.drop("Area.1", axis=1, inplace=True)
pais.drop("Population density.1", axis=1, inplace=True)
pais.drop(0, axis=0, inplace=True)
pais.rename({"Country, territory or dependency" : "nome", "Population" : "populacao", "Area" : "area"}, axis=1, inplace=True)
pais = pais.sort_values("nome")
capital.drop("Notes", axis=1, inplace=True)
capital.rename({"Country/Territory" : "nome", "City/Town" : "capital"}, axis=1, inplace=True)
capital.sort_values("nome", inplace=True)
pais = pais.join(capital.set_index('nome'), on='nome')
pais['capital'] = pais['capital'].str.replace("'", "''")
pais.update(pais[['nome', 'capital']].applymap("'{}'".format))
pais['populacao'] = pais['populacao'].str.replace(",", "")
pais['area'] = pais['area'].str.replace(",", "")
pais['Population density'] = pais['Population density'].str.replace(",", "")
pais.iloc[88]

nome                  'Gibraltar (BOT)'
populacao                         32669
area                                  6
Population density                 5445
capital                           'nan'
Name: 5, dtype: object

In [287]:
DataframeToCsv(pais, "cooked-data/pais.sql", "Pais", "(nome, populacao, area, densidadePop, capital)")